# Check if everything is in place

## Weaviate Python Client v4
> This notebook was created with Weaviate `1.30` and the Weaviate Client `4.12`

Run the below command to check if you run the latest version of the Weaviate Python Client v4.

In [ ]:
!pip show weaviate-client

## Get keys and urls

> You can update your env variables in the `.env` file at the root of the project.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

weaviate_url = os.getenv("WEAVIATE_URL")
openai_key = os.getenv("OPENAI_API_KEY")
openai_url = os.getenv("OPENAI_URL")

print(weaviate_url, openai_key, openai_url)

## Connect to Weaviate

In [ ]:
import weaviate

# Connect to the local instance
client = weaviate.connect_to_local(
    host=weaviate_url
)

client.is_ready()

## Display the available modules

> You should be able to see 'generative-openai' and 'text2vec-openai', plus many other modules.

In [ ]:
client.get_meta()

### Close the client
When you are done with the client, you should close it to release the resources.

In [ ]:
client.close()